[In English](https://marieevelb.github.io/jupyter-notebook/Arthur/)  
&nbsp;

![ECCC logo](https://eccc-msc.github.io/open-data/img_eccc-logo.png) &nbsp;

[TdM](https://eccc-msc.github.io/open-data/readme_fr/) > [Aperçu de l'utilisation](https://eccc-msc.github.io/open-data/usage/readme_fr/) > Outil d'aide à la décision en Python  

# Cas d'utilisation : utiliser les données ouvertes du Service météorologique du Canada pour l'aide à la décision

## Sommaire

Les données des services web géospatiaux [GeoMet du Service météorologique du Canada](https://eccc-msc.github.io/open-data/msc-geomet/readme_fr/) peuvent aisément être intégrées à des outils d'aide à la décision afin de faciliter la prise de décisions critiques. Cet exemple de cas d'utilisation vous montrera comment extraire et traiter les données météorologiques brutes à l'aide du langage de programmation [Python](https://www.python.org/) afin de créer un outil d'aide à la décision basé sur la notion du rapport coût-perte.

## Création d'un outil d'aide à la décision

Prenons la situation suivante: un propriétaire de crèmerie nommé Arthur souhaite planifier le nombre d'employés qu'il y aura chaque jour à la crèmerie à l'aide des prévisions du Service météorologique du Canada. En effet, comme les journées chaudes de l'été amènent beaucoup plus de clients que les journées plus froides, il souhaite utiliser un outil d'aide à la décision afin de déterminer quel nombre d'employés lui permettra de maximiser ses profits chaque jour. Selon ses calculs, la crèmerie accueille 300 clients en moyenne les journées où la température est de 25°C ou plus alors qu'elle en accueille seulement 150 les journées où la température est inférieure à 25°C. La crèmerie ne nécessite normalement qu'un seul employé, mais un deuxième est nécessaire pour réussir à servir l'ensemble des clients lorsqu'il y a plus de 200 clients dans une journée.

Afin de déterminer le nombre d'employés à mettre à l'horaire, Arthur va comparer la probabilité que la température soit égale ou supérieure à 25°C au rapport coût-perte de faire travailler un deuxième employé. Pour ce faire, la première étape est d'effectuer une requête avec le [standard Web Map Service (WMS)](https://eccc-msc.github.io/open-data/msc-geomet/web-services_fr/#web-map-service-wms) du Service météorologique du Canada afin d'obtenir la probabilité que la température soit de 25°C ou plus pour l'heure, la date et le lieu désiré à l'aide du langage de programmation Python. Pour réaliser cette étape, il faut tout d'abord importer les modules Python nécessaires et déterminer les paramètres de la requête comme la couche de données désirée, le lieu et les informations nécessaires pour obtenir la date et l'heure de chaque prédiction disponible.

La couche qui sera utilisée est celle du [Système régional de prévision d'ensemble (SRPE)](https://eccc-msc.github.io/open-data/msc-data/nwp_reps/readme_reps_fr/) qui contient la probabilité en pourcentage que la température maximale soit égale ou supérieure à 25°C (``REPS.DIAG.24_T8.ERGE25``). La période pour laquelle les prédictions sont disponibles et le pas de temps entre les prédictions a été obtenu en consultant les métadonnées de la couche à l'aide de la requête GetCapabilities suivante: https://geo.weather.gc.ca/geomet?SERVICE=WMS&VERSION=1.3.0&REQUEST=GetCapabilities&LAYERS=REPS.DIAG.24_T8.ERGE25.

In [1]:
# Importation des modules Python 
from datetime import datetime, timedelta
import re

from owslib.wms import WebMapService

# Choix des paramètres
layer = 'REPS.DIAG.24_T8.ERGE25'  # Couche de données désirée
y, x = 43.555, -73.55  # Coordonnées de l'endroit pour lequel on souhaite obtenir des prévisions
time_zone = -4 # Fuseau horaire local. Dans cet exemple, le fuseau horaire est UTC-04:00
interval = 12 # Pas de temps entre les prédictions
total_time = 48 # Nombre d'heures total de la période de prédiction
# start_time = 'AAAA-MM-JJ HH:00:00' 
# Si date et heure locale de départ spécifique, utiliser start_timee en enlevant le # 
# et en remplaçant les lettres en majuscule par les bonnes valeurs

La portion de code suivante permet d'obtenir le jour et l'heure des premières prédictions disponibles à partir du moment actuel considérant que pour cette couche de données, les prédictions disponibles sont celles des heures 00:00:00 UTC±00:00 et 12:00:00 UTC±00:00 et que les premières disponibles sont celles de demain.

In [2]:
now = datetime.now().replace(microsecond=0)

if now < now.replace(hour = 12, minute = 0, second = 0):
    time = [now.replace(hour = 0, minute = 0, second = 0) + timedelta(days = 1)]
else:
    time = [now.replace(hour = 12, minute = 0, second = 0) + timedelta(days = 1)]

Si l'on souhaite obtenir des prédictions à des dates et des heures locales spécifiques au lieu de toutes les prédictions disponibles, il faut accorder une valeur à la variable ``start_time`` dans les paramètres de la requête et enlever le ``#`` qui précède la ligne de code suivante. Dans notre exemple, Arthur souhaite obtenir toutes les prédictions disponibles, donc cette ligne de code n'est pas nécessaire.

In [3]:
# time = [datetime.strptime(start_time, '%Y-%m-%d %H:%M:%S') + timedelta(hours = 0 - time_zone)]

Maintenant que l'heure de la première prédiction a été déterminée, la portion de code suivante va permettre de calculer la date et l'heure de chaque prédiction désirée ainsi que le paramètre ``bbox`` de la requête (lieu visé par la requête).

In [4]:
# Calcul de la date et l'heure de chaque prédiction
while time[len(time) - 1] != (time[0] + timedelta(hours = total_time)):
    time.append(time[len(time) - 1] + timedelta(hours = interval))
    
# Calcul du paramètre bbox
min_x, min_y, max_x, max_y = x - 0.25, y - 0.25, x + 0.25, y + 0.25

La requête peut maintenant être envoyée au WMS. Comme la réponse est un fichier texte contenant différentes informations, il faut ensuite extraire la probabilité que la température soit égale ou supérieure à 25°C du résultat de la requête. Afin d'obtenir la probabilité pour chaque date et heure désirée, une boucle est utilisée afin de refaire la requête pour chaque temps de prédiction déterminé précédemment. 

In [5]:
# Connexion au WMS
wms = WebMapService('https://geo.weather.gc.ca/geomet?SERVICE=WMS&REQUEST=GetCapabilities', version = '1.3.0')

# Boucle pour effectuer les requêtes et extraire les probabilités
info = []
pixel_value = []
prediction_index = 0 # Position de la prédiction dans la liste time
while prediction_index < len(time):
    info.append(wms.getfeatureinfo(layers = [layer],
                       srs = 'EPSG:4326',
                       bbox = (min_x, min_y, max_x, max_y),
                       size = (100, 100),
                       format = 'image/jpeg',
                       query_layers = [layer],
                       info_format = 'text/plain',
                       xy = (50, 50),
                       feature_count = 1,
                       time = str(time[prediction_index].isoformat()) + 'Z'
                       ))
    # Extraction des probabilités
    text = info[prediction_index].read().decode('utf-8')
    pixel_value.append(float(str(re.findall("value_0\s=\s\W\W*\d*.*\d+", text)).strip('[""]').rpartition("'")[2]))
    print(f'Pour la date et l\'heure {time[prediction_index].strftime("%m/%d/%Y %H:%M:%S")},' +
        f' la probabilité que t°>=25°C est de {str(pixel_value[prediction_index])}%.')
    prediction_index += 1

Pour la date et l'heure 05/12/2020 12:00:00, la probabilité que t°>=25°C est de 0.0%.
Pour la date et l'heure 05/13/2020 00:00:00, la probabilité que t°>=25°C est de 0.0%.
Pour la date et l'heure 05/13/2020 12:00:00, la probabilité que t°>=25°C est de 0.0%.
Pour la date et l'heure 05/14/2020 00:00:00, la probabilité que t°>=25°C est de 0.0%.
Pour la date et l'heure 05/14/2020 12:00:00, la probabilité que t°>=25°C est de 0.0%.


Une fois les probabilités obtenues, Arthur doit calculer le rapport coût-perte de faire travailler un deuxième employé à la crèmerie. Comme mentionné plus tôt, Arthur sait qu'en moyenne, la crèmerie accueille 300 clients lorsque la température est de 25°C ou plus et qu'un employé n'est en mesure de servir que 200 clients par jour. Arthur a également calculé qu'un client lui rapporte en moyenne 3,50\\$. Chaque employé a un salaire horaire de 10$ et travaille 8 heures par jour, soit de l'ouverture à la fermeture de la crèmerie. Le coût représente donc le salaire du deuxième employé et les pertes représentent les profits perdus dus aux clients qui n'ont pas pu être servis par un seul employé.

In [6]:
#Données de départ
clients_above25 = 300 # Nombre de clients lorsque la température est de 25°C ou plus
clients_served = 200 # Chaque employé peut servir 200 clients/jour
profit_per_client = 3.5 # Un client rapporte en moyenne 3,50$
salary = 10 # Les employés sont payés 10,00$/h
hours_per_day = 8 # Les enployés travaillent 8h par jour

# Coût pour ajouter un deuxième employé pendant une journée
cost = hours_per_day * salary

# Profits perdus si un seul employé est présent une journée où la température est supérieure à 25°C
losses = (clients_above25-clients_served) * profit_per_client

# Rapport coût-perte
cost_losses_ratio = cost / losses

print(f'Le rapport coût-perte est de {str(round(cost_losses_ratio, 2))}.')


Le rapport coût-perte est de 0.23.


Afin de prendre une décision, Arthur veut comparer la perte moyenne engendrée si un seul employé travaille au coût que représente l'ajout un employé supplémentaire. La perte moyenne se calcule de la manière suivante:

$$Pertes * Probabilité$$

Pour que ce soit avantageux de faire travailler un deuxième employé, la perte moyenne si un seul employé travaille doit être supérieure au coût pour faire travailler un employé de plus donc: 

$$Pertes * Probabilité > Coût$$

Si l'on réarrange l'équation, on obtient: 

$$Probabilité > Coût/Pertes$$

Ainsi, si la probabilité que la température soit égale ou supérieure à 25°C est plus élevée que le rapport coût-perte, Arthur devrait faire travailler deux employés. Dans le cas contraire, il devrait en faire travailler un seul.


In [7]:
prediction_index = 0
while prediction_index < len(time):
    if pixel_value[prediction_index] > cost_losses_ratio:
        print(f'Pour la date et l\'heure {time[prediction_index].strftime("%m/%d/%Y %H:%M:%S")},' +
              f' il est recommandé d\'avoir 2 employés.')
    else:
        print(f'Pour la date et l\'heure {time[prediction_index].strftime("%m/%d/%Y %H:%M:%S")},' +
              f' il est recommandé d\'avoir un seul employé.')
    prediction_index += 1

Pour la date et l'heure 05/12/2020 12:00:00, il est recommandé d'avoir un seul employé.
Pour la date et l'heure 05/13/2020 00:00:00, il est recommandé d'avoir un seul employé.
Pour la date et l'heure 05/13/2020 12:00:00, il est recommandé d'avoir un seul employé.
Pour la date et l'heure 05/14/2020 00:00:00, il est recommandé d'avoir un seul employé.
Pour la date et l'heure 05/14/2020 12:00:00, il est recommandé d'avoir un seul employé.


## Conclusion

Grâce à cet outil d'aide à la décision, Arthur sait maintenant combien d'employés il devrait faire travailler chaque jour afin de maximiser les profits de sa crèmerie. En changeant les paramètres de la requête et les conditions déterminant quelle décision doit être prise, il est possible d'adapter le code Python de cet exemple pour répondre à vos besoins.